In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Input 
from tensorflow.keras import callbacks
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import h5py
import tensorflow 
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#config = tensorflow.ConfigProto(allow_soft_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#sess=tensorflow.Session(config=config)
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


In [ ]:
#DATA PREPROCESSING

INIT_LR = 1e-4
EPOCHS = 75
BS = 10
train_data_dir = '/gdrive/My Drive/Colab Notebooks/dataset/train'
validation_data_dir = '/gdrive/My Drive/Colab Notebooks/dataset/test'
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rotation_range=15,
        fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=BS,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=BS,
        class_mode='categorical')

Found 186 images belonging to 2 classes.
Found 92 images belonging to 2 classes.


In [ ]:
def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
   # x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
   # x = BatchNormalization(name='bn3')(x)
   # x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
   # x = BatchNormalization(name='bn4')(x)
   # x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    # x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(128, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(2, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model
model =  build_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      [(None, 224, 224, 3)]     0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
Conv2_1 (SeparableConv2D)    (None, 112, 112, 128)     8896      
_________________________________________________________________
Conv2_2 (SeparableConv2D)    (None, 112, 112, 128)     17664     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)      

In [ ]:
#initialision cov2 weights from vgg16 imagenet 
f = h5py.File('/gdrive/My Drive/Colab Notebooks/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r')

# Select the layers for which you want to set weight.

w,b = f['block1_conv1']['block1_conv1_W_1:0'], f['block1_conv1']['block1_conv1_b_1:0']
model.layers[1].set_weights = [w,b]

w,b = f['block1_conv2']['block1_conv2_W_1:0'], f['block1_conv2']['block1_conv2_b_1:0']
model.layers[2].set_weights = [w,b]

w,b = f['block2_conv1']['block2_conv1_W_1:0'], f['block2_conv1']['block2_conv1_b_1:0']
model.layers[4].set_weights = [w,b]

w,b = f['block2_conv2']['block2_conv2_W_1:0'], f['block2_conv2']['block2_conv2_b_1:0']
model.layers[5].set_weights = [w,b]

f.close()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      [(None, 224, 224, 3)]     0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
Conv2_1 (SeparableConv2D)    (None, 112, 112, 128)     8896      
_________________________________________________________________
Conv2_2 (SeparableConv2D)    (None, 112, 112, 128)     17664     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)      

In [ ]:
#Training phase
opt = Adam(lr=0.0001, decay=1e-5)
es = callbacks.EarlyStopping(patience=18)
chkpt = callbacks.ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=False)
model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)
history =model.fit_generator(training_set,steps_per_epoch=186//BS,epochs=EPOCHS,validation_data=test_set,validation_steps=94//BS,callbacks=[es,chkpt])


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/75
18/18 [==============================] - ETA: 0s - loss: 0.6790 - accuracy: 0.6250WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: best_model_todate/assets
18/18 [==============================] - 62s 3s/step - loss: 0.6790 - accuracy: 0.6250 - val_loss: 0.6711 - val_accuracy: 0.7333
Epoch 2/75
18/18 [==============================] - 8s 449ms/step - loss: 0.5899 - accuracy: 0.7216 - val_loss: 0.7609 - val_accuracy: 0.2556
Epoch 3/75
18/18 [==========

In [ ]:
import tensorflow as tf
tf.keras.models.save_model(model, 'my_model3.hdf5')

In [ ]:
! pip install streamlit

     |████████████████████████████████| 7.4MB 16.0MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 163kB 54.0MB/s 
     |████████████████████████████████| 4.5MB 50.5MB/s 
     |████████████████████████████████| 112kB 51.8MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 122kB 60.1MB/s 
  Created wheel for watchdog: filename=watchdog-1.0.1-cp36-none-any.whl size=72206 sha256=8a9a2ef1181312ce2db42561bca512a877e0e54e81a856b3a72ead81f0ccf15d
  Stored in directory: /root/.cache/pip/wheels/d8/ce/d8/31a48288b5728794feda5ac479fa324cc1cde4398c29eff064
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=ac5fd11084933fb8bc6e13df76325fc159940f11bac3e2ab30b93dc36f7e48a8
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built watchdog blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=

In [ ]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18781 sha256=0e679cbddff5e1758800ce0dd94731ed1409bb3d2f29f710f76aa041d5033f87
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf

st.set_option('deprecation.showfileUploaderEncoding', False)
@st.cache(allow.output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/my_model3.hdf5')
  return model
model = load_model()
st.write("""
         # Covid-19 Chest X-Ray Classification
         """
         )
file = st.file_uploader("Upload X-Ray", type=["jpg","png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
def import_and_predict(image_data, model):
  
  
  testimg1=image.img_to_array(image_data)
  testimg1=np.expand_dims(testimg1,axis=0)
  testimg1= testimg1/255.0
  result=model.predict(testimg1)
  return result
  
if file is None:
  st.text("Please upload an image file")
else:
  image = Image.open(file)
  st.image(image, use_column_width = True)
  predictions = import_and_predict(image, model)
  st.success(predictions)

Writing app.py


In [ ]:
!ngrok authtoken 1lb8FTI46c53Aw30WVeZstIUXOo_6cpvf2nhZXwbaHvwehn9w

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
!pgrep streamlit

1395


In [ ]:
from pyngrok import ngrok
publ_url = ngrok.connect(port=8501)
publ_url

<NgrokTunnel: "http://9e03386bacf5.ngrok.io" -> "http://localhost:80">

In [ ]:
from tensorflow.keras.preprocessing import image
testimg=image.load_img("/gdrive/My Drive/Colab Notebooks/1-s2.0-S1684118220300608-main.pdf-001.jpg",target_size=(224, 224))
testimg=image.img_to_array(testimg)
testimg=np.expand_dims(testimg,axis=0)
testimg= testimg/255.0
result1=model.predict(testimg)
print(result1)
print(training_set.class_indices)

[[0.9771672 0.0228328]]
{'covid': 0, 'normal': 1}
